In [7]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [8]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Add the file to SparkFiles and read it into a DataFrame
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
file_path = SparkFiles.get("home_sales_revised.csv")

# Read the file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the DataFrame schema
df.printSchema()


root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [16]:
df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date=datetime.date(2022, 4, 8), date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76)

In [10]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [11]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query3 = spark.sql("""
    SELECT YEAR(date) AS Year, ROUND(AVG(price), 2) AS Avg_Price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY YEAR(date)
""")
query3.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [17]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

# Register DataFrame as a temporary view
df.createOrReplaceTempView("home_sales")

# Execute SQL query
query4 = spark.sql("""
    SELECT date_built AS Year, ROUND(AVG(price), 2) AS Avg_Price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")
query4.show()


+----+---------+
|Year|Avg_Price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [27]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
from pyspark.sql.functions import year, to_date

# Convert the 'date_built' column to date type
df = df.withColumn("date_built", to_date(df["date_built"].cast("string"), "yyyy"))

# Register DataFrame as a temporary view
df.createOrReplaceTempView("home_sales")

# Execute SQL query to find the average price of homes meeting the criteria
query = """
    SELECT YEAR(date_built) AS Year, ROUND(AVG(price), 2) AS Avg_Price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY YEAR(date_built)
    ORDER BY YEAR(date_built)
"""

result = spark.sql(query)
result.show()



+----+---------+
|Year|Avg_Price|
+----+---------+
|2010|285010.22|
|2011|276553.81|
|2012|307539.97|
|2013|303676.79|
|2014|298264.72|
|2015|297609.97|
|2016| 293965.1|
|2017|280317.58|
+----+---------+



In [28]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

from pyspark.sql.functions import round

# Start measuring execution time
start_time = time.time()

# Execute the query to find the "view" rating for homes with prices greater than or equal to $350,000
query = """
    SELECT ROUND(AVG(view), 2) AS Avg_View
    FROM home_sales
    WHERE price >= 350000
"""

result = spark.sql(query)
result.show()

# Calculate and print the execution time
print("--- %s seconds ---" % (time.time() - start_time))


+--------+
|Avg_View|
+--------+
|   32.26|
+--------+

--- 3.4417901039123535 seconds ---


In [29]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: date, date_built: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [30]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [31]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

# Run the query on the cached DataFrame
query = """
    SELECT view, ROUND(AVG(price), 2) AS Avg_Price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
"""

result_cached = spark.sql(query)
result_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))



+----+----------+
|view| Avg_Price|
+----+----------+
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  81|1053472.79|
|  76|1058802.78|
|  91|1137372.73|
|  93|1026006.06|
|  52| 733780.26|
|  86|1070444.25|
|  94| 1033536.2|
|  57|  734340.5|
|  54| 798684.82|
|  96|1017815.92|
|  92| 970402.55|
|  64| 767036.67|
|  61| 746877.59|
|  88|1031719.35|
|  72| 780914.67|
|  59|  791453.0|
+----+----------+
only showing top 20 rows

--- 6.892840385437012 seconds ---


In [38]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

df.write.partitionBy("date_built").parquet("/Users/limu/Downloads/home_sales_challenge/parquet/")

parquet_path = "/Users/limu/Downloads/home_sales_challenge/parquet/"
df_partitioned = spark.read.parquet(parquet_path)

df_partitioned.createOrReplaceTempView("partitioned_home_sales")


In [40]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet("/Users/limu/Downloads/home_sales_challenge/parquet/")
parquet_df

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: date]

In [44]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView("parquet_df")

In [45]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

from pyspark.sql.functions import round

start_time = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS Avg_Price
    FROM parquet_df
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
"""

result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|Avg_Price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 5.9908058643341064 seconds ---


In [47]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [48]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
if not is_cached:
    print("The 'home_sales' table is no longer cached.")
else:
    print("The 'home_sales' table is still cached.")

The 'home_sales' table is no longer cached.
